In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv(r'/kaggle/input/global-wheat-detection/train.csv')

In [ ]:
df.head()

In [ ]:
df.shape

This csv file contain image_id, width, height, bbox, source. There are total of 147793 data-entry in file.but most of data is releted to single image. which contains number of wheat in image.It also given bounding box for each wheat. and image height and width is given.

In [ ]:
df.info()

In [ ]:
print(df['source'].isnull().sum())
print(df['source'].unique())

There are total 7 sources of data.
1. usask_1
2. arvalis_1
3. inrae_1
4. ethz_1
5. arvalis_3
6. rres_1
7. arvalis_2

In [ ]:
df['source'].value_counts()

Above are the total number of observations per source. but in single image there are multipal bounding boxes for wheats. so its not much usefull.

In [ ]:
len(df['image_id'].unique())

Traning dataset contain total of 3373 images.

In [ ]:
print(df['width'].unique())
print(df['height'].unique())



Each image size is 1024* 1024

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# import useful tools
from glob import glob
from PIL import Image
import cv2

# import data visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

from bokeh.plotting import figure
from bokeh.io import output_notebook, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, Panel
from bokeh.models.widgets import Tabs

# import data augmentation
import albumentations as albu

# Global Wheat Detection Competition EDA

Detection of wheat spikes with computer vision opens a lot of opportunity for the farmers and breeders like:

controlling the growth stage of the plants on the field: the number oand the area of spikes is raising closer to the harvest date;
controlling the heatlth of the plants: unusually small number or small size of plants might be a signal of deceased plants;
spikes density characteristic and approximate yield estimation for different varieties of wheat.
In this notebook I am exploring the data and giving some thoughts on what to pay attention when making and validating the models.

# General Dataset Information

Let's just look at the numbers first. Those numbers will give us a hint which images might be interesting to look at.
1 The number of train and test images:

In [ ]:
# Setup the paths to train and test images
TRAIN_DIR = '/kaggle/input/global-wheat-detection/train/'
TEST_DIR = '/kaggle/input/global-wheat-detection/test/'
TRAIN_CSV_PATH = '/kaggle/input/global-wheat-detection/train.csv'

# Glob the directories and get the lists of train and test images
train_fns = glob(TRAIN_DIR + '*')
test_fns = glob(TEST_DIR + '*')

Compute at the number of train and test images:

In [ ]:
print('Number of train images is {}'.format(len(train_fns)))
print('Number of test images is {}'.format(len(test_fns)))

We have only 10 test images here, other test images will be used to evaluate the prediction models during the submission.

Just 3422 images for the training seems to be not much at all. Data augmentation techniques will be definetely required in this competition.

2. The number of bounding boxes (wheat spikes) per image:

Construct dataframe with all images (images with no bboxes will have nan values in all columns except image_id):

In [ ]:
# Load the dataframe with the bounding boxes
train = pd.read_csv(TRAIN_CSV_PATH)

In [ ]:
train.head()

In [ ]:
# Create a dataframe with all train images
all_train_images = pd.DataFrame([fns.split('/')[-1][:-4] for fns in train_fns])
all_train_images.columns=['image_id']
print(all_train_images.head())
print(all_train_images.shape)

In [ ]:
# Merge all train images with the bounding boxes dataframe
all_train_images = all_train_images.merge(train, on='image_id', how='left')
#print(all_train_images.shape)
print(all_train_images.head())

In [ ]:
# replace nan values with zeros
all_train_images['bbox'] = all_train_images.bbox.fillna('[0,0,0,0]')

In [ ]:
# split bbox column
bbox_items = all_train_images.bbox.str.split(',', expand=True)
all_train_images['bbox_xmin'] = bbox_items[0].str.strip('[ ').astype(float)
all_train_images['bbox_ymin'] = bbox_items[1].str.strip(' ').astype(float)
all_train_images['bbox_width'] = bbox_items[2].str.strip(' ').astype(float)
all_train_images['bbox_height'] = bbox_items[3].str.strip(' ]').astype(float)

In [ ]:
all_train_images.head()

In [ ]:
print('{} images without wheat heads.'.format(len(all_train_images) - len(train)))

In [ ]:
bbox_items

Let's plot some image examples:


In [ ]:
def get_all_bboxes(df, image_id):
    image_bboxes = df[df.image_id == image_id]
    
    bboxes = []
    for _,row in image_bboxes.iterrows():
        bboxes.append((row.bbox_xmin, row.bbox_ymin, row.bbox_width, row.bbox_height))
        
    return bboxes

def plot_image_examples(df, rows=3, cols=3, title='Image examples'):
    fig, axs = plt.subplots(rows, cols, figsize=(10,10))
    for row in range(rows):
        for col in range(cols):
            idx = np.random.randint(len(df), size=1)[0]
            img_id = df.iloc[idx].image_id
            
            img = Image.open(TRAIN_DIR + img_id + '.jpg')
            axs[row, col].imshow(img)
            
            bboxes = get_all_bboxes(df, img_id)
            
            for bbox in bboxes:
                rect = patches.Rectangle((bbox[0],bbox[1]),bbox[2],bbox[3],linewidth=1,edgecolor='r',facecolor='none')
                axs[row, col].add_patch(rect)
            
            axs[row, col].axis('off')
            
    plt.suptitle(title)

In [ ]:
plot_image_examples(all_train_images)

We can see images taken at different lighting conditions and plant maturity stages!

Count number of bounding boxes per image:

In [ ]:
# compute the number of bounding boxes per train image
all_train_images['count'] = all_train_images.apply(lambda row: 1 if np.isfinite(row.width) else 0, axis=1)
train_images_count = all_train_images.groupby('image_id').sum().reset_index()

In [ ]:
all_train_images.head()

In [ ]:
train_images_count.head()

In [ ]:
def hist_hover(dataframe, column, colors=["#94c8d8", "#ea5e51"], bins=30, title=''):
    hist, edges = np.histogram(dataframe[column], bins = bins)
    
    hist_df = pd.DataFrame({column: hist,
                             "left": edges[:-1],
                             "right": edges[1:]})
    hist_df["interval"] = ["%d to %d" % (left, right) for left, 
                           right in zip(hist_df["left"], hist_df["right"])]

    src = ColumnDataSource(hist_df)
    plot = figure(plot_height = 400, plot_width = 600,
          title = title,
          x_axis_label = column,
          y_axis_label = "Count")    
    plot.quad(bottom = 0, top = column,left = "left", 
        right = "right", source = src, fill_color = colors[0], 
        line_color = "#35838d", fill_alpha = 0.7,
        hover_fill_alpha = 0.7, hover_fill_color = colors[1])
        
    hover = HoverTool(tooltips = [('Interval', '@interval'),
                              ('Count', str("@" + column))])
    plot.add_tools(hover)
    
    output_notebook()
    show(plot)

In [ ]:
hist_hover(train_images_count, 'count', title='Number of wheat spikes per image')

Most of the images have 20-50 wheat spikes on them.

Let's plot some examples with small number of spikes per image:

In [ ]:
less_spikes_ids = train_images_count[train_images_count['count'] < 10].image_id
plot_image_examples(all_train_images[all_train_images.image_id.isin(less_spikes_ids)], title='Example images with small number of spikes')

These are some very strange examples:

1. on some of the images all we see is the ground;
2. some of the images are just zoomed in a lot.


Plot the images with many spikes:

In [ ]:
many_spikes_ids = train_images_count[train_images_count['count'] > 100].image_id
plot_image_examples(all_train_images[all_train_images.image_id.isin(many_spikes_ids)], title='Example images with large number of spikes')

3. Area of bounding boxes:

In [ ]:
# compute bounding box areas
all_train_images['bbox_area'] = all_train_images['bbox_width'] * all_train_images['bbox_height']

In [ ]:
# plot a histogram of bounding box areas
hist_hover(all_train_images, 'bbox_area', title='Area of a single bounding box')

The maximum area of bounding box:

In [ ]:
all_train_images.bbox_area.max()

The distribution of individual areas of bounding boxes has a very long tail. It would be interesting to look at the images with those large bounding boxes.

Let's plot some examples of large bounding boxes:

In [ ]:
large_boxes_ids = all_train_images[all_train_images['bbox_area'] > 200000].image_id
plot_image_examples(all_train_images[all_train_images.image_id.isin(large_boxes_ids)], title='Example images with large bbox area')

What are these anomally large bounding boxes?? I think those should be removed while training!!!

Similarly, let's look at very small bounding boxes:

In [ ]:
min_area = all_train_images[all_train_images['bbox_area']> 0].bbox_area.min()
print('The smallest bounding box area is {}'.format(min_area))

In [ ]:
small_boxes_ids = all_train_images[(all_train_images['bbox_area'] < 50) & (all_train_images['bbox_area'] > 0)].image_id
plot_image_examples(all_train_images[all_train_images.image_id.isin(small_boxes_ids)], title='Example images with large bbox area')

If you look very close, you can probably see those tinyest bounding boxes near the corners and borders of the images. Probably, the boundries were drawn first, than the images were cut into several ones. That is why we see those strange small bounsing boxes in the corners.

It is not necessary to clean these, because they won't have much effect on the IOU metric.

4. Area of bounding boxes per image:

In [ ]:
from PIL import Image
import numpy as np
from skimage import io

In [ ]:
img = io.imread('/kaggle/input/global-wheat-detection/train/a1ecd6a63.jpg')
plt.imshow(img)
#b6ab77fd7[834.0, 222.0, 56.0, 36.0]	
#826.0	371.0	102.0	132.0
#765.0	580.0	121.0	129.0	

In [ ]:
#print(img)
img1 = img[580:580+129, 765:765+121, :]
#img11 = img1.resize((28,28))
plt.imshow(img1)

In [ ]:
img2 = img[371:371+132, 826:826+102, :]
#gray_img2 = img2.convart('L')
plt.imshow(img2)

In [ ]:
img3 = img[312:312+118,465:465+119,:]
plt.imshow(img3)
#465.0	312.0	119.0	118.0